In [ ]:
import os
import shutil
import random
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Importing tensorflow and its utilities
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import numpy as np
#from tensorflow.python.keras.applications.resnet50 import preprocess_input
from keras.applications import *
#from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import load_img, img_to_array

##Visualising the data

In [ ]:
base_bone_dir = os.path.join('/content/')
age_df = pd.read_csv(os.path.join(base_bone_dir, 'boneage-training-dataset.csv' ))
age_df['path'] = age_df['id'].map(lambda x: os.path.join(base_bone_dir,
                                                         'boneage-training-dataset',
                                                         'boneage-training-dataset',
                                                         '{}.png'.format(x)))
age_df.drop(age_df.tail(8000).index, inplace = True)
age_df['exists'] = age_df['path'].map(os.path.exists)
print(age_df['exists'].sum(), 'images found of', age_df.shape[0], 'total')  #checking if the allocated path exists or not
age_df['gender'] = age_df['male'].map(lambda x: 'male' if x else 'female')
boneage_mean = age_df['boneage'].mean()
boneage_div = 2*age_df['boneage'].std() # it's for normalization (multiplication by 2). if you don't want it you have to remove the following #

# If we don't want normalization
#boneage_mean = 0
#boneage_div = 1.0

age_df['boneage_zscore'] = age_df['boneage'].map(lambda x: (x-boneage_mean)/boneage_div)
age_df.dropna(inplace = True)  #dropping the null values
age_df.sample(5)

4611 images found of 4611 total


,id,boneage,male,path,exists,gender,boneage_zscore
3398,5342,120,True,/content/boneage-training-dataset/boneage-trai...,True,male,-0.096656
3868,5863,144,False,/content/boneage-training-dataset/boneage-trai...,True,female,0.176279
1913,3513,138,True,/content/boneage-training-dataset/boneage-trai...,True,male,0.108045
2126,3744,168,True,/content/boneage-training-dataset/boneage-trai...,True,male,0.449213
4404,6449,156,True,/content/boneage-training-dataset/boneage-trai...,True,male,0.312746


##Loading data from ImageDataGenerator

In [ ]:
train_df = pd.read_csv("/content/boneage-training-dataset.csv")
# quadratic approach
quadratic = True
image_size = 256

# some kind of ratio if not quadratic approach
img_rows = image_size if quadratic else 144
img_cols = image_size if quadratic else 114


def read_and_prep_images(img_paths, img_height, img_width, color_mode="grayscale"):
    imgs = [load_img(img_path, target_size=(img_height, img_width), color_mode=color_mode) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    #output = preprocess_input(img_array)
    return(img_array)


train_df['imagepath'] = [f'{pid}.png' for pid in train_df.id]
#Lazy way of creating training and test set.
train = train_df.head(10000)
test = train_df.tail(2600)


datagen=ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.15,
                           width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                           horizontal_flip=True, fill_mode="nearest")
train_generator=datagen.flow_from_dataframe(dataframe=train,
                                            directory="/content/boneage-training-dataset/boneage-training-dataset/",
                                            x_col="imagepath",
                                            y_col="boneage",
                                            class_mode="other",
                                            target_size=(img_rows,img_cols),
                                            batch_size=32)

valid_generator=datagen.flow_from_dataframe(dataframe=test,
                                            directory="/content/boneage-training-dataset/boneage-training-dataset/",
                                            x_col="imagepath",
                                            y_col="boneage",
                                            class_mode="other",
                                            target_size=(img_rows,img_cols),
                                            batch_size=32)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

Found 10000 validated image filenames.
Found 2600 validated image filenames.


##Importing VGG19

In [ ]:
vgg19 = tf.keras.applications.VGG19()

574710816/574710816 [==============================] - 3s 0us/step


In [ ]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

##Customization for Use Case

In [ ]:
from tensorflow.keras import layers
# Removiand adding our output layer
x = vgg19.layers[-5].output
x = tf.keras.layers.Conv2D(7, 3, strides=1, padding='same')(x)
x = Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(2, strides=2)(x)
x = tf.keras.layers.Conv2D(3, 3, strides=1, padding='same')(x)
x = Activation('relu')(x)
x = tf.keras.layers.Conv2D(2, 3, strides=1, padding='same')(x)
x = Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(2, strides=2)(x)
#x = tf.keras.layers.Flatten()(x)
x = Dense(units=512, activation='linear')(x)
x = Dense(units=256, activation='linear')(x)
x = Dense(units=64, activation='linear')(x)


output = Dense(units=1, kernel_regularizer=l2(0.01), activation='linear')(x)

x = vgg19.layers[-5].output
x = Dense(units=512, activation='linear')(x)
x = Dense(units=256, activation='linear')(x)
x = Dense(units=64, activation='linear')(x)
output = Dense(units=1, kernel_regularizer=l2(0.01), activation='linear')(x)

In [ ]:
x = vgg19.layers[-5].output
output = Dense(units=1, kernel_regularizer=l2(0.01), activation='linear')(x)

In [ ]:
model = Model(inputs=vgg19.input, outputs=output)

In [ ]:
from keras.metrics import mean_absolute_error
def mae_months(in_gt, in_pred):
    return mean_absolute_error(boneage_div*in_gt, boneage_div*in_pred)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='hinge', metrics=[mae_months, 'mae'])

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights_custom.h5".format('bone_age')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min', save_weights_only = True)


reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=8) #patience of 5 because limit in terms of resources, if you can increase the values.
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=valid_generator, validation_split=0.2, batch_size=200, verbose=1  )

##Saving the model

In [ ]:
model.save('\content\boneage_custom_vgg.h5')

In [ ]:
from sklearn.metrics import mean_absolute_error as sk_mae

pred_Y = boneage_div*model.predict(test_X, batch_size = 32, verbose = True)+boneage_mean
test_Y_months = boneage_div*test_Y+boneage_mean
print("Mean absolute error on test data: "+str(sk_mae(test_Y_months,pred_Y)))

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (6,6))
ax1.plot(test_Y_months, pred_Y, 'r.', label = 'predictions')
ax1.plot(test_Y_months, test_Y_months, 'b-', label = 'actual')
ax1.legend()
ax1.set_xlabel('Actual Age (Months)')
ax1.set_ylabel('Predicted Age (Months)')

In [ ]:
ord_idx = np.argsort(test_Y)
ord_idx = ord_idx[np.linspace(0, len(ord_idx)-1, 8).astype(int)] # take 8 evenly spaced ones
fig, m_axs = plt.subplots(4, 2, figsize = (16, 32))
sum = 0
counter = 0
for (idx, c_ax) in zip(ord_idx, m_axs.flatten()):
    c_ax.imshow(test_X[idx, :,:,0], cmap = 'bone')

    c_ax.set_title('Age: %2.1fY\nPredicted Age: %2.1fY' % (test_Y_months[idx]/12.0,
                                                           pred_Y[idx]/12.0))
    temp_test = test_Y_months[idx]/12.0
    temp_pred = pred_Y[idx]/12.0
    sum = sum + (temp_test - temp_pred)
    counter = counter + 1
    c_ax.axis('off')
fig.savefig('trained_img_predictions.png', dpi = 300)
sum = sum/counter
print("Average Deviation : " , sum)